<a href="https://www.kaggle.com/code/tanishksarode/image-classification-using-deep-and-hand-crafted-f?scriptVersionId=232853988" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Image Classification using Deep and Hand-Crafted Feature Modeling
1. Extract deep features from the pre-trained VGG-16 model and extract hand crafted
Histogram Oriented Gradient (HOG) features for MNIST dataset. Stack the deep
features with HOG features and model it using a random forest classifier to classify
the MNIST dataset. Run the hybrid model 5 times and compute the mean accuracy.
2. Extract deep features from the pre-trained VGG-16 model and extract hand crafted
Scale Invariant Feature Transform (SIFT) features for MNIST dataset. Stack the deep
features with SIFT features and model it using a random forest classifier to classify
the MNIST dataset. Run the hybrid model 5 times and compute the mean accuracy.
3. Extract deep features from the pre-trained VGG-16 model and extract hand crafted
SIFT and HOG features for MNIST dataset. Stack the deep features with HOG and
SIFT features and model it using a random forest classifier to classify the MNIST
dataset. Run the hybrid model 5 times and compute the mean accuracy.
4. Extract deep features from the pre-trained VGG-16 model and extract hand crafted
SIFT and HOG features for MNIST dataset. Stack the deep features with HOG and
SIFT features and use PCA to transform and reduce the dimension (Test using
different component values). Model the transformed features using a random forest
classifier to classify the MNIST dataset. Run the hybrid model 5 times and compute
the mean accuracy.
5. Draw conclusions on the best model among the above four models for classifying
MNIST dataset.

In [1]:
from tensorflow.keras.datasets import mnist
import cv2
import numpy as np


(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train, y_train), (X_test, y_test) = (X_train[:500], y_train[:500]), (X_test[:100], y_test[:100])

X_train_resized = np.array([cv2.resize(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB), (224, 224)) for img in X_train])
X_test_resized = np.array([cv2.resize(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB), (224, 224)) for img in X_test])



11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Load VGG-16
vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Create a feature extractor model
model = Model(inputs=vgg16.input, outputs=vgg16.output)

# Extract features
deep_train_features = model.predict(X_train_resized)
deep_test_features = model.predict(X_test_resized)

# Flatten features
deep_train_features = deep_train_features.reshape(len(X_train), -1)
deep_test_features = deep_test_features.reshape(len(X_test), -1)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 723ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step


In [3]:
from skimage.feature import hog

def extract_hog_features(images):
    hog_features = [hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True) for img in images]
    return np.array(hog_features)

hog_train_features = extract_hog_features(X_train)
hog_test_features = extract_hog_features(X_test)


In [4]:
import cv2
from sklearn.cluster import KMeans

sift = cv2.SIFT_create()

def extract_sift_features(images, num_clusters=50):
    descriptors_list = []
    valid_indices = []  # Track indices of images with descriptors

    for i, img in enumerate(images):
        keypoints, descriptors = sift.detectAndCompute(img, None)
        if descriptors is not None:
            descriptors_list.append(descriptors)
            valid_indices.append(i)  # Store valid image index

    if len(descriptors_list) == 0:
        return np.zeros((len(images), num_clusters))  # Return zeros if no descriptors exist

    # Stack descriptors and apply KMeans
    all_descriptors = np.vstack(descriptors_list)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    kmeans.fit(all_descriptors)

    # Assign each image a histogram of cluster frequencies
    sift_features = []
    for descriptors in descriptors_list:
        labels = kmeans.predict(descriptors)
        hist = np.histogram(labels, bins=np.arange(num_clusters + 1))[0]
        sift_features.append(hist)

    sift_features = np.array(sift_features)

    # Handle missing images by adding zero vectors
    full_sift_features = np.zeros((len(images), num_clusters))
    full_sift_features[valid_indices] = sift_features  # Fill valid indices

    return full_sift_features

sift_train_features = extract_sift_features(X_train)
sift_test_features = extract_sift_features(X_test)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def train_and_evaluate(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Model 1: VGG + HOG
X_train_1 = np.hstack((deep_train_features, hog_train_features))
X_test_1 = np.hstack((deep_test_features, hog_test_features))
acc_1 = train_and_evaluate(X_train_1, X_test_1, y_train, y_test)

# Model 2: VGG + SIFT
X_train_2 = np.hstack((deep_train_features, sift_train_features))
X_test_2 = np.hstack((deep_test_features, sift_test_features))
acc_2 = train_and_evaluate(X_train_2, X_test_2, y_train, y_test)

# Model 3: VGG + HOG + SIFT
X_train_3 = np.hstack((deep_train_features, hog_train_features, sift_train_features))
X_test_3 = np.hstack((deep_test_features, hog_test_features, sift_test_features))
acc_3 = train_and_evaluate(X_train_3, X_test_3, y_train, y_test)

# Model 4: VGG + HOG + SIFT + PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=100)  # Tune this value
X_train_4 = pca.fit_transform(X_train_3)
X_test_4 = pca.transform(X_test_3)
acc_4 = train_and_evaluate(X_train_4, X_test_4, y_train, y_test)

# Compute Mean Accuracy over 5 runs
import numpy as np

def compute_mean_accuracy(X_train, X_test, y_train, y_test, runs=5):
    accuracies = [train_and_evaluate(X_train, X_test, y_train, y_test) for _ in range(runs)]
    return np.mean(accuracies)

mean_acc_1 = compute_mean_accuracy(X_train_1, X_test_1, y_train, y_test)
mean_acc_2 = compute_mean_accuracy(X_train_2, X_test_2, y_train, y_test)
mean_acc_3 = compute_mean_accuracy(X_train_3, X_test_3, y_train, y_test)
mean_acc_4 = compute_mean_accuracy(X_train_4, X_test_4, y_train, y_test)

# Print final results
print(f"Mean Accuracy (VGG + HOG): {mean_acc_1}")
print(f"Mean Accuracy (VGG + SIFT): {mean_acc_2}")
print(f"Mean Accuracy (VGG + HOG + SIFT): {mean_acc_3}")
print(f"Mean Accuracy (VGG + HOG + SIFT + PCA): {mean_acc_4}")


Mean Accuracy (VGG + HOG): 0.9399999999999998
Mean Accuracy (VGG + SIFT): 0.9399999999999998
Mean Accuracy (VGG + HOG + SIFT): 0.9800000000000001
Mean Accuracy (VGG + HOG + SIFT + PCA): 0.97
